In [215]:
# Include necessary imports
import os
import torch 
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
import torch.nn.functional as F

In [216]:
# Pre Process Data
def get_pitch_class(note):
    return note % 12

def find_matching_octave_note(df):
    bass_line = df.iloc[3].values
    last_bass_note = bass_line[-1]
    return last_bass_note

def explore_for_lowest_tonic(df, pitch_class):
    bass_notes = df.iloc[3].values  
    matching_notes = [note for note in bass_notes if get_pitch_class(note) == pitch_class]
    if matching_notes:
        return min(matching_notes) 
    else:
        return bass_notes[0]

def detect_tonic(df):
    candidate_note = find_matching_octave_note(df)
    pitch_class = get_pitch_class(candidate_note)
    true_tonic_note = explore_for_lowest_tonic(df, pitch_class)
    return true_tonic_note

def key_transposition(df):
    tonic_note = detect_tonic(df)
    transpose_val = 48 - tonic_note
    df = (df + transpose_val).clip(lower=0, upper=127)
    return df


def normalize_df(df):
    X_std = df / 127
    return X_std

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                
                transposed_df = key_transposition(df.transpose())
                normalized_df = normalize_df(transposed_df)
                if dirname == 'test':
                    test.append(normalized_df)
                if dirname == 'train':
                    train.append(normalized_df)
                if dirname == 'valid':
                    validation.append(normalized_df)

# Model

In [217]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=100, n_layers=2, dropout_rate=.3):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.layer_norms = torch.nn.ModuleList([torch.nn.LayerNorm(hidden_dim) for _ in range(n_layers)])
        self.fc = torch.nn.Linear(hidden_dim, output_size * 128)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device),
                torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device))

    def forward(self, x, target=None, hidden=None, teacher_forcing=0.5):
        batch_size, seq_len, _ = x.size()

        if hidden is None:
            hidden = self.init_hidden(batch_size)

        lstm_output, hidden = self.lstm(x, hidden)

        for layer_idx in range(self.n_layers):
            lstm_output = self.layer_norms[layer_idx](lstm_output)

        model_output = self.fc(lstm_output)
        model_output = model_output.view(batch_size, seq_len, 3, 128)

        outputs = []

        for t in range(seq_len):
            model_output_step = model_output[:, t, :, :]  # [batch_size, 3, 128]

            if target is not None and random.random() < teacher_forcing:
                x = torch.tensor([2, 0, 1])
#                 print(F.one_hot((target[:, t, :] * 128).squeeze().to(torch.int64), num_classes=128))
                next_input = F.one_hot((target[:, t, :] * 128).to(torch.int64), num_classes=128).float()  
#                 print(next_input)# [batch_size, 3, 128]
#                 print("teacher forcing next input: ", next_input)
            else:
#                 pred_indices = model_output_step  # [batch_size, 3, 1]
#                 next_input = F.one_hot((pred_indices).to(torch.int64), num_classes=128).float()  # [batch_size, 3, 128]
                next_input = model_output_step
#                 print("next input for the model: ", next_input)
#                 print("model output next input: ", next_input)

            outputs.append(next_input.unsqueeze(1))

        output = torch.cat(outputs, dim=1)
        output = output.view(batch_size * seq_len * 3, 128)
#         print("OUTPUT IS", outputs)
        return output

# Train

In [224]:
def train_model(model, optimizer, criterion, num_epochs):
    model.train()
    for song_index, song in enumerate(train[:5]):
        print(f"Training on song {song_index + 1}")
        
        melody = torch.tensor(song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, song.shape[1], 1)
        harmonies = torch.tensor(song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
#         harmonies_with_zero = torch.zeros(1, song.shape[1], 3)
#         melody_with_empty_harmonies = torch.cat((melody, harmonies_with_zero), dim = -1)
        harmonies_for_loss = harmonies_to_class(harmonies)
       
        
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            ratio = .9 - (.9 - .1) * (epoch / num_epochs)
            ratio = max(.1, ratio)
            output = model(x=melody, target=harmonies, teacher_forcing=ratio)
#             output = output.reshape(-1, 128)
            harmonies_for_loss = harmonies_for_loss.reshape(-1)
            output = output.view(-1, 128)  # [batch_size * seq_len * 3, 128]
            batch_size, seq_len, _ = melody.size()
            output = output.view(batch_size * seq_len * 3, 128)
#             print("output is ", output)
#             print("harmonies are: ", harmonies)
#             target = harmonies.view(-1).long()
#             print("target is ", harmonies_for_loss)
            loss = criterion(output, harmonies_for_loss)

#             with torch.no_grad():
#                 print(f"Targets    : {harmonies_for_loss[:50]}")
#                 print(f"Predictions: {output[:50].argmax(dim=1)}")
#             print("Output is ", output)
#             print("Harmonies for loss is ", harmonies_for_loss)
#             loss = criterion(output, harmonies_for_loss)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
                    

        model.eval()
        total_val_loss = 0
        

        
        # validation songs
        with torch.no_grad():
            for val_song in validation:
                val_melody = torch.tensor(val_song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, val_song.shape[1], 1)
                val_harmonies = torch.tensor(val_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
                val_harmonies = harmonies_to_class(val_harmonies)
                val_harmonies_with_zero = torch.zeros(1, val_song.shape[1], 3)
                val_melody_with_empty_harmonies = torch.cat((val_melody, val_harmonies_with_zero), dim=-1)
                
                val_output = model(val_melody)
                val_output = val_output.reshape(-1, 128)
                val_harmonies = val_harmonies.reshape(-1)
                
                val_loss = criterion(val_output, val_harmonies)
                total_val_loss += val_loss.item()

        average_val_loss = total_val_loss / len(validation)
        print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

In [ ]:
def harmonies_to_class(harmonies):
    harmonies_classes = torch.round(harmonies * 127).long()  
    return harmonies_classes

criterion = torch.nn.CrossEntropyLoss()
num_epochs = 100

model = Model(1, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
train_model(model, optimizer, criterion, num_epochs)

Training on song 1


# Hyperparameter Tuning

In [ ]:
learning = [0.01, .001]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000, 10000]
best_loss = float('inf')
best_params = {}

for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [220]:
def inverse_df(df):
    X_scaled = df * 127
    return X_scaled

def midi_to_note(part):
    print("PART:", part[0])
    result = stream.Part()
    count = 1
    prev = round(part[0])
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

# def midi_to_note_harmonies(part):
#     probabilities = torch.softmax(torch.tensor(part), dim=-1).numpy()
# #     print(probabilities)
#     result = stream.Part()
#     count = 1
#     predicted_notes = np.softmax(part, axis=1)
#     print(predicted_notes)
#     prev = predicted_notes[0]
#     for i in range(1, len(part)):
#         curr = predicted_notes[i]
#         if curr == prev:
#             count += 1
#         else:
#             this_note = note.Note(prev, quarterLength=count/4)
#             result.append(this_note)
#             count = 1
#         prev = curr
#     this_note = note.Note(prev, quarterLength=count/4)
#     result.append(this_note)
#     return result

def output_to_sheet_music(melody, result):
    melody = inverse_df(melody).squeeze()
    inversed = result.T
    
    score = stream.Score()
    melody_part = midi_to_note(melody.numpy())
    
    alto_notes = result[0::3].numpy()  # Start at 0, take every 3rd element
    tenor_notes = result[1::3].numpy() # Start at 1, take every 3rd element
    bass_notes = result[2::3].numpy()
    
    print(tenor_notes)
    
    alto_part = midi_to_note(alto_notes)
    tenor_part = midi_to_note(tenor_notes)
    bass_part = midi_to_note(bass_notes)

    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [223]:
test_song = test[0]
melody = test_song.iloc[0]

melody = torch.tensor(test_song.iloc[0].values, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
harmonies = torch.tensor(test_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
harmonies_with_zero = torch.zeros(1, test_song.shape[1], 3)

model_input = torch.cat((melody, harmonies_with_zero), dim = -1)
result = model(x=melody)
predictions = result.argmax(dim=-1)
print(predictions)
output_to_sheet_music(melody, predictions)

tensor([67, 64, 60, 67, 64, 60, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [ ]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss

In [ ]:
# weights get zeroed out
# vanishing gradient
# not adjusting the weights; gradient is zero?